In [1]:
import os
from dotenv import load_dotenv
from agents import Agent, Runner

load_dotenv()

api_key = os.getenv("OPENAI_API_KEY")
if not api_key:
    raise ValueError("OPENAI_API_KEY environment variable not set")

In [5]:
agent = Agent(
    name="Test Agent",
    instructions="A helpful assistant that can perform tasks.",

)
result =await Runner.run(agent, "What is the capital of France?")
print(result.final_output)

The capital of France is Paris.


In [9]:
from agents import Agent,Runner,ModelSettings
from dotenv import load_dotenv

load_dotenv()

# define detailed instructions for weather assistant

weather_instructions="""
You are a weather information assistant who helps user understand weather patterns and phenomena.

YOUR EXPERTISE:
 - Explaining weather concepts and terminology
 - Describing how different weather systems work
 - Answering questions about climate and seasonal patterns
 - Explaining the science behind weather events

LIMITATIONS:
- You cannot provide real time weather forecasts for specific locations
- You don't have the current weather data
- You should not make predictions about future weather events

STYLE:

- Use clear , accessible language that non-meterologists can understand
- Include interesting weather facts when relevant
- Be enthusiastic about meteorology and climate science
"""

# create specialised weather assistant
weather_assistant= Agent(
    name ='WeatherAssist',
    instructions=weather_instructions,
    model="gpt-4.1-mini",
    model_settings= ModelSettings(
        temperature=0.5,
        max_tokens=256
    )
)

In [10]:
result= await Runner.run(
    weather_assistant,"can you tell me about the relationship between climate change and extreme weather events"
)

print(result.final_output)

Absolutely! The relationship between climate change and extreme weather events is a fascinating and important topic in meteorology.

Climate change refers to long-term shifts in temperature, precipitation, and other atmospheric conditions on Earth, primarily driven by increased greenhouse gases like carbon dioxide from human activities. These changes affect the patterns and intensity of weather events.

Here’s how climate change influences extreme weather:

1. **Warmer Temperatures Increase Heatwaves:** As the planet warms, heatwaves become more frequent, intense, and longer-lasting. This happens because higher average temperatures push the extremes upward.

2. **More Intense Storms:** Warmer ocean waters fuel tropical storms and hurricanes, providing more energy that can lead to stronger winds and heavier rainfall. This means storms can become more destructive.

3. **Changes in Rainfall Patterns:** Some regions experience heavier rainfall and flooding, while others may face droughts. 

In [12]:
from agents import Agent, Runner,WebSearchTool
from dotenv import load_dotenv

load_dotenv()

# create a research assistant with web search capabilities
research_assistant= Agent(
    name = "Research Assistant",
    instructions = """
    You're a research assistant who helps users find accurate and up-to-date information on various topics.
    1. Search the web for relevant information using the web search tool.
    2. Synthesize the information you find into a coherent response.
    3. Cite your sources when providing information.
    4. If you cannot find the information, be honest about it.
    5. Structure your responses clearly and logically and in bullet points when appropriate.
    6. Use formal and academic language.
    """,
    tools=[WebSearchTool()],
    model="gpt-4.1-mini",
)

async def research_topic(topic):
    result = await Runner.run(
        research_assistant, f"Please research and summarize the latest findings on the topic: {topic}"
    )
    return result.final_output

summary=await research_topic("Advancements in AI Agens technologies")
print(summary)


Recent advancements in AI agent technologies have led to significant developments across various sectors, enhancing automation, efficiency, and user experience. Key progressions include:

**1. Integration of AI Agents into Business Platforms**

- **Microsoft 365 Copilot**: Microsoft has fully integrated AI agents into Microsoft Teams, SharePoint, and Viva Engage. These agents function proactively, setting meeting agendas, organizing files, guiding discussions, and monitoring task progress, thereby enhancing teamwork and productivity. ([windowscentral.com](https://www.windowscentral.com/artificial-intelligence/microsoft-copilot/microsoft-365-copilot-ai-agents-reach-new-milestone?utm_source=openai))

- **Salesforce's Agentforce**: Salesforce introduced Agentforce, a suite of generative AI agents designed for sales, marketing, commerce, and customer service roles. Powered by the Atlas reasoning engine, these agents aim to operate autonomously within defined parameters, reducing the need f

In [1]:
import os
import requests
from datetime import datetime
from typing import Optional, List
from dataclasses import dataclass

from agents import Agent, Runner, function_tool
from dotenv import load_dotenv

load_dotenv()


@dataclass
class WeatherInfo:
   temperature: float
   feels_like: float
   humidity: int
   description: str
   wind_speed: float
   pressure: int
   location_name: str
   rain_1h: Optional[float] = None
   visibility: Optional[int] = None


@function_tool
def get_weather(lat: float, lon: float) -> str:
   """Get the current weather for a specified location using OpenWeatherMap API.

   Args:
       lat: Latitude of the location (-90 to 90)
       lon: Longitude of the location (-180 to 180)
   """
   # Get API key from environment variables
   WEATHER_API_KEY = os.getenv("OPENWEATHERMAP_API_KEY")

   # Build URL with parameters
   url = f"https://api.openweathermap.org/data/2.5/weather?lat={lat}&lon={lon}&appid={WEATHER_API_KEY}&units=metric"

   try:
       response = requests.get(url)
       response.raise_for_status()
       data = response.json()

       # Extract weather data from the response
       weather_info = WeatherInfo(
           temperature=data["main"]["temp"],
           feels_like=data["main"]["feels_like"],
           humidity=data["main"]["humidity"],
           description=data["weather"][0]["description"],
           wind_speed=data["wind"]["speed"],
           pressure=data["main"]["pressure"],
           location_name=data["name"],
           visibility=data.get("visibility"),
           rain_1h=data.get("rain", {}).get("1h"),
       )

       # Build the response string
       weather_report = f"""
       Weather in {weather_info.location_name}:
       - Temperature: {weather_info.temperature}°C (feels like {weather_info.feels_like}°C)
       - Conditions: {weather_info.description}
       - Humidity: {weather_info.humidity}%
       - Wind speed: {weather_info.wind_speed} m/s
       - Pressure: {weather_info.pressure} hPa
       """
       return weather_report

   except requests.exceptions.RequestException as e:
       return f"Error fetching weather data: {str(e)}"

In [2]:
# Create a weather assistant
weather_assistant = Agent(
   name="Weather Assistant",
   instructions="""You are a weather assistant that can provide current weather information.
  
   When asked about weather, use the get_weather tool to fetch accurate data.
   If the user doesn't specify a country code and there might be ambiguity,
   ask for clarification (e.g., Paris, France vs. Paris, Texas).
  
   Provide friendly commentary along with the weather data, such as clothing suggestions
   or activity recommendations based on the conditions.
   """,
   tools=[get_weather]
)

In [5]:
async def main():
    runner=Runner()
    simple_request=await runner.run(weather_assistant, "What's your capabilties?")
    request_with_location= await runner.run(weather_assistant, "What's the weather in Noida?")
    print(simple_request.final_output)
    print("---"*70)
    print(request_with_location.final_output)

await main()

I’m your weather assistant! Here’s what I can do for you:

- **Current Weather Info:** I can provide up-to-date weather conditions (temperature, wind, humidity, etc.) for any location in the world.
- **Clothing & Activity Tips:** I’ll offer friendly advice on what to wear and fun activities you can enjoy based on the current weather.
- **Travel Weather Checks:** Planning a trip? I can help you check the weather at multiple destinations.
- **Global Assistance:** I handle ambiguous city names by making sure I get the correct country or region (e.g., Paris, France vs. Paris, Texas).

If you’d like to know the weather somewhere, just tell me the city (and country or state if needed), and I’ll get you the current report—with some helpful tips! How can I help you today?
------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
The current we